### Important libraries

In [37]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import imagenet_utils
import imutils
from imutils.object_detection import non_max_suppression
import pandas as pd
import time
import cv2
import numpy as np

In [38]:
def sliding_window(image, step, ws):
    for y in range(0, image.shape[0]-ws[1], step):
        for x in range(0, image.shape[1]-ws[0], step):
            yield(x, y, image[y:y+ws[1], x:x+ws[0]])

In [39]:
def image_pyramid(image, scale=1.5, minSize=(224, 224)):
    yield image
    while True:
        w = int(image.shape[1]/scale)
        image = imutils.resize(image, width=w)
        if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
            break
        yield image

In [40]:
width = 600
pyr_scale = 1.5
win_step = 16
roi_size = (224, 224)
input_size = (224, 224)

In [41]:
model = MobileNet(weights='imagenet', include_top=True)

### Loading the image to be detected

In [42]:
original_img = cv2.imread('tiger.jpg')

### Image resizing

In [49]:
original = cv2.imread('tiger.jpg')
original = imutils.resize(original, width=width)
(H, W) = original.shape[:2]

### Regions with location

In [50]:
pyramid = image_pyramid(original, scale=pyr_scale, minSize=roi_size)
for i in pyramid:
    print(i.shape)

(774, 600, 3)
(516, 400, 3)
(343, 266, 3)


In [51]:
rois = []
locs = []
pyramid = image_pyramid(original, scale=pyr_scale, minSize=roi_size)
for image in pyramid:
    scale = W/float(image.shape[1])    
    for (x,y,roiOrig) in sliding_window(image,win_step,roi_size):
        x=float(x*scale)
        y=float(y*scale)
        w=float(roi_size[0]*scale)
        h=float(roi_size[1]*scale)
        roi=cv2.resize(roiOrig,input_size)
        roi=img_to_array(roi)
        roi=preprocess_input(roi)
        
        rois.append(roi)
        locs.append([x,y,x+w,y+h])        

### Probability of image in regions

In [52]:
rois = np.array(rois, dtype='float32')
preds = model.predict(rois)
preds=imagenet_utils.decode_predictions(preds,top=1)

### Non-Maximum-Supression

In [53]:
df=pd.DataFrame(columns=['lab','prob','bb'])
for index,i in enumerate(preds):
    if float(i[0][2])>0.9:
        df=df.append({'lab':i[0][1],'prob':i[0][2],
                      'bb':locs[index]},ignore_index=True)
       
       
boxes1=[]
obj=[]
for i in df['lab'].unique():
    nms=df[df['lab']==i]
    boxes = non_max_suppression(np.array(nms['bb'].tolist()), nms['prob'].tolist())
    boxes1.append(boxes)
    obj.append(i)

### Object Detection with bounding box

In [55]:
clone = original.copy()
for i in range(len(boxes1)):
    bb=boxes1[i][0]
    cv2.rectangle(clone, (bb[0], bb[1]), (bb[2], bb[3]),(0, 255, 0), 2)
    y = bb[1] - 10 if bb[1] - 10 > 10 else bb[1] + 10
    cv2.putText(clone, obj[i], (bb[0], y),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
    cv2.imshow("Final Objects", clone)
    cv2.waitKey(0);cv2.destroyAllWindows()
